In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
import re
import stopwordsiso
from stopwordsiso import stopwords
# stopwords(["zh"])  # Chinese
import jieba
from sklearn.preprocessing import MultiLabelBinarizer

In [21]:
# clean text and seg
def preprocessingTextFull(text, sep = ' '):
    text = text.lower()
    text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
    text = re.sub(r'<.*?>', '', text) #remove html
#     text = re.sub(r'[0-9]+', '', text) #remove number
#     text = re.sub(r'[^\w\s]', '', text) #remove punctiation
    text = re.sub("[\@\-\;\>\<\:\?\.\!\/_,$%^(\"\']+" , ' ' , text) #remove punctiation, keep ****

    # remove stopword
    stop_words = list(stopwords(["zh"]))
    more_s = ['请问','谢谢您','谢谢你''谢谢','您好','_']
    stop = stop_words + more_s
    text = "".join([word for word in text if word not in stop]) #remove stopwords
    
    for c in ['\r', '\n', '\t'] :
        text = re.sub(c, ' ', text) #replace newline and tab with tabs\
        text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
#         text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    text_cut = sep.join(jieba.cut(text, cut_all=False))
        
    return text_cut

def preprocessingTextPartly(text, sep = ' '):
    
    text_cut = sep.join(jieba.cut(text, cut_all=False))
        
    return text_cut

# more_s = ['请问', '没', 'kiehls', 'linepaymoney','谢谢您','谢谢你''谢谢','您好', '姓名','元', '电话', '手机', 'line', 'pay', 'money','不能', '一下', '需要','linepay', '今天', '现在', '最近','_','公司','point','没有']


#     text = re.sub('[^\u4e00-\u9fa5]+', ' ', text) # remove ASCII strings
#   text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings


In [33]:
df = pd.read_csv('training_sim.csv')
df = df.rename(columns={'sim_label': 'label','sim_detail_label': 'detail_label','sim_text': 'text'})
df = df.dropna()
df.head()

,label,detail_label,text
0,行销活动问题,未收到活动回馈金问题,"您好,我于2020/7/6 下午5:38左右绑定兆丰银行并开启自动储值功能,但仍未收到88元..."
1,行销活动问题,未收到活动点数问题,刷屈臣氏480元，却无点数回馈?
2,行销活动问题,未收到活动点数问题,已经确认过，你们只有给4月跟6月喔\n\n5月的点数是漏发的，5月帐单缴费13075元\n\...
3,行销活动问题,未收到活动点数问题,请问我有另外一个手机号码用一卡通缴电费为什么都没有回馈点谢谢
4,行销活动问题,未收到活动点数问题,Hi \n\n１.我的同意行销条款是有打勾（绿色勾勾）\n\n 2. 姓名：林*炜\n 身份...


In [34]:
df.index.names = ['id']

In [35]:
df['label'] = df['label'].str.strip()
df['detail_label'] = df['detail_label'].str.strip()

In [36]:
df1 = df.drop(df[df['label']  == '未成年注册问题'].index)

In [37]:
df = df1.drop(df[df['label']  == '境外合作'].index)

In [38]:
df.shape

(6947, 3)

In [39]:
df.reset_index(drop = True)

,label,detail_label,text
0,行销活动问题,未收到活动回馈金问题,"您好,我于2020/7/6 下午5:38左右绑定兆丰银行并开启自动储值功能,但仍未收到88元..."
1,行销活动问题,未收到活动点数问题,刷屈臣氏480元，却无点数回馈?
2,行销活动问题,未收到活动点数问题,已经确认过，你们只有给4月跟6月喔\n\n5月的点数是漏发的，5月帐单缴费13075元\n\...
3,行销活动问题,未收到活动点数问题,请问我有另外一个手机号码用一卡通缴电费为什么都没有回馈点谢谢
4,行销活动问题,未收到活动点数问题,Hi \n\n１.我的同意行销条款是有打勾（绿色勾勾）\n\n 2. 姓名：林*炜\n 身份...
...,...,...,...
6942,注册一卡通帐户问题,未成年注册问题,分类:LINE Pay Money>注册问题>未成年注册问题\n您的大名:王*程\n电子邮件...
6943,储值问题,连结银行问题,进线台湾银行连结失败
6944,注册一卡通帐户问题,金融验证问题,您好，我目前所遇到之问题为在注册阶段即收到已有绑定帐户而无法进行会员注册，但我所收到之简讯为...
6945,注册一卡通帐户问题,简讯验证码问题,分类:LINE Pay Money>注册问题>简讯验证码问题\n您的大名:曾*筠\n电子邮件...


In [40]:
df['text'] = df['text'].apply(preprocessingTextFull)
df['label'] = df['label'].apply(preprocessingTextPartly)
df['detail_label'] = df['detail_label'].apply(preprocessingTextPartly)

data = df.dropna().reset_index(drop = True)

In [41]:
data.isnull().sum()

label           0
detail_label    0
text            0
dtype: int64

In [42]:
# from sklearn.preprocessing import LabelEncoder
# item_encoder = LabelEncoder()
# df['label_encode'] = item_encoder.fit_transform(df.label)
# df.head()

In [43]:
df = df.sample(frac = 1, random_state = 123)
df['node ids'] = df.index

In [44]:
df.head()

,label,detail_label,text
id,,,
0,行销 活动 问题,未 收到 活动 回馈 金 问题,2020 7 6 午 5 38 左右 绑定 兆丰 银行 开 启动 储值功...
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 480 元数 回馈
2,行销 活动 问题,未 收到 活动 点数 问题,确认 46 喔 5 数漏 发 5 帐单 缴费 13075 元 应道 13075 * ...
3,行销 活动 问题,未 收到 活动 点数 问题,问外 手机号码 卡通 缴 电费 没 回馈 谢谢
4,行销 活动 问题,未 收到 活动 点数 问题,hi 意 行销 条款 勾 绿色 勾勾 2 姓名 林 * 炜 身份证 f12 ...


In [45]:
df.isnull().sum()

label           0
detail_label    0
text            0
dtype: int64

In [46]:
# create files
# label
# df.label_encode.to_csv('label.csv', index = False)
# raw file
df.to_csv('raw_file.csv', index = 'id')


In [130]:
data_train, data_test    = train_test_split(df, test_size=0.2, random_state=1, stratify = df.label)

data_train, data_val     = train_test_split(data_train, test_size=0.25, random_state=1, ) # 0.25 x 0.8 = 0.2

In [131]:
data_train['train_idx'] = data_train.index
data_train['train_idx'].to_csv('train.csv', index = False)

data_test['test_idx'] = data_test.index
data_test['test_idx'].to_csv('test.csv', index = False)

data_val['val_idx'] = data_val.index
data_val['val_idx'].to_csv('val.csv', index = False)

<ipython-input-131-19566edc702b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['test_idx'] = data_test.index


In [127]:
print(len(data_train))
print(len(data_test))
print(len(data_val))
print(len(df))

4167
1390
1390
6947
